In [3]:
import os
from ultralytics import YOLO
import logging
logging.getLogger('matplotlib').setLevel(logging.WARNING)

In [4]:
# CONSTANTES
BATCH_SIZE = -1
C = 3 # number of channels of the input image
H = 640 # heigh of the input image
W = 640 # width of the input image
current_directory = os.getcwd()

## ONNX: Model conversion

In [ ]:
%run onnx_transform.py --weights weights/yolov8lsalmons.pt --input_shape $BATCH_SIZE $C $H $W

## TRT fp32

In [ ]:
%run build_trt.py --weights weights/yolov8lsalmons.onnx  --fp32 --input_shape $BATCH_SIZE $C $H $W --engine_name yolov8lsalmons_fp32.engine

## TRT fp16

In [ ]:
%run build_trt.py --weights weights/yolov8lsalmons.onnx  --fp16 --input_shape $BATCH_SIZE $C $H $W --engine_name yolov8lsalmons_fp16.engine

## TRT int8

In [ ]:
%run build_trt.py --weights weights/yolov8lsalmons.onnx  --int8 --input_shape $BATCH_SIZE $C $H $W --engine_name yolov8lsalmons_int8.engine

# A complex example of build trt

In [ ]:
%run build_trt.py --weights weights/yolov8lsalmons.onnx --fp16 --build_op_lvl 3 --max_batch_size 32 --dynamic_hw --input_shape -1 3 -1 -1 --engine_name yolov8lsalmons_fp16_bs32_oplvl3.engine

## validate

In [ ]:
model = YOLO('weights/yolov8lsalmons_fp16.engine', task='segment')
metrics = model.val(data='datasets/salmons/salmons.yaml', task='segment', verbose=False,conf=0.4,device='cuda')
print('base model box mAP50: ', metrics.box.map50)
print('base model box mAP50-95: ', metrics.box.map)